I plan to update the model in a timely manner. For the current situation, I referred to the following.

# **if it is useful for you, please vote！**

* https://www.kaggle.com/code/realneuralnetwork/jpx-lgbm-model-overfitting-high-score
* https://www.kaggle.com/code/paulorzp/median-model-jpx
* https://www.kaggle.com/code/realneuralnetwork/jpx-lgbm-model-overfitting-high-score
* https://www.kaggle.com/code/paulorzp/jpx-simple-overfitting-model

# import　必要なライブラリをインポート

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna
import matplotlib.pyplot as plt
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

# read dara&processing データ読み込み＆加工

In [ ]:
# prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
path = "../input/jpx-tokyo-stock-exchange-prediction/"
prices = pd.concat([pd.read_csv(f"{path}train_files/stock_prices.csv"),
                    pd.read_csv(f"{path}supplemental_files/stock_prices.csv")],
                    ignore_index=True,
                    sort=False
                  ).drop_duplicates(["Date","SecuritiesCode"], keep="first")

In [ ]:
def p_prices(prices):
    prices.Date = pd.to_datetime(prices.Date).view(int)
    prices.fillna(0,inplace=True)
    return prices

In [ ]:
NDAYS = 14
lastdays = prices[prices["Date"]>=prices.Date.iat[-2000*NDAYS]].reset_index(drop=True)

In [ ]:
lastdays = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return lastdays.loc[_id_]
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [ ]:
prices.head()

In [ ]:
prices["HLV"] = (prices["Avg"])/(prices["High"] -prices["Low"]+0.001)
prices["HLV2"] = (prices["Volume"])*(prices["High"] -prices["Low"]+0.001)
prices["HLV3"] = (prices["Volume"])*abs(prices["Open"] -prices["Close"]+0.001)
prices["HLV4"] = (prices["Volume"])/abs(prices["Open"] -prices["Close"]+0.001)
prices["HLV5"] = (prices["Volume"])*abs(prices["Open"] -prices["High"]+0.001)
prices["HLV6"] = (prices["Volume"])*abs(prices["Open"] -prices["Close"]+0.001)
prices["ratio"] = (prices["Open"]-prices["Close"])/(prices["Avg"]+0.001)
prices["ratio2"] = (prices["High"]-prices["Low"])/(prices["Avg"]+0.001)

In [ ]:
prices.Date = pd.to_datetime(prices.Date)
prices = p_prices(prices)
# prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
# X=prices.loc[:,["SecuritiesCode","High","Open","Close","Low","Volume","HLV"]]
# X=prices.loc[:,["SecuritiesCode","Avg","High","Open","Close","Low","Volume","HLV","HLV2","HLV3","HLV4","HLV5","HLV6","ratio","ratio2"]]
X=prices.loc[:,["SecuritiesCode","Avg","High","Open","Close","Low","Volume","ratio","ratio2"]]
y=prices.loc[:,["Target"]]
dtrain = xgb.DMatrix(X, label=y)

# XGBoost optimization parameter preparation＆create model　XGBoost最適化用パラメータ用意＆モデル作成

In [ ]:
params = {
        'objective': 'reg:squarederror','silent':1, 'random_state':1234, 
        'eval_metric': 'rmse',"subsample":0.5,
    }
num_round = 1000
# watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

In [ ]:
model_o = xgb.train(params,
                    dtrain,#訓練データ
                    num_round,#設定した学習回数
#                     early_stopping_rounds=20,
#                     evals=watchlist,
                    )

In [ ]:
params = {
    "silent": 1,
    "max_depth": 6,
    "min_child_weight": 1,
    "eta": 0.1,
    "tree_method": "exact",
    "objective": "reg:linear",
    "eval_metric": "mae",#rmse
    "subsample": 0.2,
    "predictor": "cpu_predictor"
}

In [ ]:
def objective(trial):

    params = {
        "silent": 1,
        "max_depth": trial.suggest_int("max_depth", 100, 2000),
        "min_child_weight": 125,
        "eta": trial.suggest_loguniform("eta", 0.2, 0.9),
        "tree_method": "exact",
        "objective": "reg:linear",
        "eval_metric": "mae",#mae,rmse
        "gamma": trial.suggest_int("gamma", 0, 10),
        "alpha": trial.suggest_int("alpha", 0, 10),
        "lambda": trial.suggest_int("lambda", 0, 10),
#         "subsample": trial.suggest_float('subsample', 0.1, 0.8),
        "predictor": "cpu_predictor"  
    }

    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=30,
        seed=0,
        nfold=7, # CVの分割数
        metrics={"rmse"},
        early_stopping_rounds=5
    )

#     return cv_results["test-rmse-mean"].min()

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

In [ ]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
params["max_depth"] = trial.params["max_depth"]
params["eta"] = trial.params["eta"]

model_o = xgb.train(params=params,
                  dtrain=dtrain,
                  num_boost_round=15,
                  early_stopping_rounds=5,
#                   evals=[(dtest, "test")])
                    evals=[(dtrain, "train")])

In [ ]:
# model_o=LGBMRegressor(num_leaves=1000, learning_rate=0.2, n_estimators=200)
model_o.fit(X,y)
model_o.score(X,y)

# API connection & data processing & prediction　API接続＆データ加工＆予測

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    prices = p_prices(prices)
    df = sample_prediction[["SecuritiesCode"]]
    df["Avg"] = prices["SecuritiesCode"].apply(get_avg)
    prices["Avg"] = df["Avg"]
#     df.Date = pd.to_datetime(df.Date)
#     df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    df["High"] = prices["High"]
    df["Open"] = prices["Open"]
    df["Close"] = prices["Close"]
    df["Low"] = prices["Low"]
    df["Volume"] = prices["Volume"]
#     df["AdjustmentFactor"] = prices["AdjustmentFactor"]
#     df["ExpectedDividend"] = prices["ExpectedDividend"]
#     df["SupervisionFlag"] = prices["SupervisionFlag"]
    df["HLV"] = (prices["Avg"])/(prices["High"] -prices["Low"]+0.001)
    df["HLV2"] = (prices["Volume"])*(prices["High"] -prices["Low"]+0.001)
    df["HLV3"] = (prices["Volume"])*abs(prices["Open"] -prices["Close"]+0.001)
    df["HLV4"] = (prices["Volume"])/abs(prices["Open"] -prices["Close"]+0.001)
    df["HLV5"] = (prices["Volume"])*abs(prices["Open"] -prices["High"]+0.001)
    df["HLV6"] = (prices["Volume"])*abs(prices["Open"] -prices["Close"]+0.001)
    df["ratio"] = (prices["Open"] -prices["Close"])/(prices["Avg"]+0.001)
    df["ratio2"] = (prices["High"] -prices["Low"])/(prices["Avg"]+0.001)
#     df["HLV"] = prices["HLV"]
    df = xgb.DMatrix(df)
    sample_prediction["Prediction"] = model_o.predict(df)
    sample_prediction["rate"] = sample_prediction["Prediction"]
    sample_prediction = sample_prediction.sort_values(by = "rate", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    sample_prediction.drop(["rate"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)